## Import all the relevant Modules 

In [ ]:
%matplotlib inline


#TODO: Rename module shortcuts such as hb into something usefull after development
import Modules.M05_Dynamical_Correlations.Module_Dynamical_Hubbard as dh
from Modules.M00_General.Module_Widgets_and_Sliders import Save_Figure_Button, Click_Save_Figure, set_filename

# default Jupyter widgets
import ipywidgets as widgets
from ipywidgets import HBox, VBox

# Plotting
import seaborn as sns
sns.set_style("darkgrid")
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size':14})

# for printlenght and save figures button
import functools
import numpy as np
np.set_printoptions(linewidth=150) #set output length, default=75

def close_widgets(DIR) -> None:
    """Close all widgets `wi` and displays `di` in notebook directory `dir()`.
    Also clear `Save_Figure_Button`
    """
    for i in range(100):
        if f"w{i}" in DIR:
            exec(f"w{i}.close()")
        if f"d{i}" in DIR:
            exec(f"d{i}.close()")
    # clear `Save_Figure_Button` callbacks, otherwise all previous callbacks are executed and figures are saved multiple times
    Save_Figure_Button._click_handlers.callbacks = []
            

%load_ext autoreload
%autoreload 2

# Dynamical Correlation Functions
<!---  Define a few convenience macros for bra-ket notation. -->
$\newcommand{\ket}[1]{\left\vert{#1}\right\rangle}$
$\newcommand{\bra}[1]{\left\langle{#1}\right\vert}$
$\newcommand{\braket}[2]{\left\langle{#1}\middle|{#2}\right\rangle}$
$\newcommand{\dyad}[2]{\left|{#1}\middle\rangle\middle\langle{#2}\right|}$
$\newcommand{\mel}[3]{\left\langle{#1}\vphantom{#2#3}\right|{#2}\left|{#3}\vphantom{#1#2}\right\rangle}$
$\newcommand{\expval}[1]{\left\langle{#1}\right\rangle}$
$\newcommand\dif{\mathop{}\!\mathrm{d}}$
$\newcommand\ii{\mathrm{i}}$
$\newcommand{\coloneqq}{\mathop{:=}}$
$\newcommand{\abs}[1]{\left\vert{#1}\right\vert}$
$\newcommand{\vb}[1]{\mathbf{#1}}$
$\newcommand{\im}[1]{\operatorname{Im}{#1}}$
$\newcommand{\re}[1]{\operatorname{Re}{#1}}$

Up until now we only looked at equal-time correlation functions. We want to further look at so-called **dynamical correlation functions** which depend on time or frequency. To this end we define the time dependent correlation function
$$ \large
    \expval{\hat{A}(t)\hat{B}(0)} \coloneqq \mel{\psi_g}{e^{\ii t \hat{H}} \hat{A} e^{-\ii t \hat{H}}  \hat{B}}{\psi_g},
$$
where $\hat{A}$ and $\hat{B}$ are arbitrary operators and $\hat{A}(t) \coloneqq e^{\ii t \hat{H}} \hat{A} e^{-\ii t \hat{H}}$ is the time evolution of the operator $\hat{A}$.


## Spectral Representation of the Correlation Function
By using the completeness relation of the Hamiltonian's eigenfunctions $\ket{n}$, i.e. $1 = \sum_n \dyad{n}{n}$ and the eigenvalue equation for $H$, we can rewrite the correlation function as

$$ \large
\begin{align*} 
	\expval{\hat{A}(t)\hat{B}(0)} &\coloneqq \mel{\psi_g}{e^{\ii t \hat{H}} \hat{A} e^{-\ii t \hat{H}}  \hat{B}}{\psi_g} \\
	%
	&= \sum_n  \mel{\psi_g}{e^{\ii t \hat{H}} \hat{A} \dyad{n}{n} e^{-\ii t \hat{H}}  \hat{B}}{\psi_g} \\
	%
	&= \sum_n e^{-\ii t (E_n - E_g)} \mel{\psi_g}{ \hat{A} }{n}\mel{n} {\hat{B}}{\psi_g} \\
	%
	&= \sum_n e^{-\ii t \bar{E}_n} \mel{\psi_g}{ \hat{A} }{n}\mel{n} {\hat{B}}{\psi_g},
\end{align*}
$$

where we defined the energy difference $\bar{E}_n \coloneqq E_n - E_g$.
Note that we drastically simplified the expression from the complicated exponential of an operator to an exponential of just a number.
As we have the sum of exponentials, it is natural to look at the Fourier transform of the correlation function.

## Fourier Transform of the Correlation Function
If one were to naively apply the Fourier transform from time domain $t$ to frequency domain $\omega$ of the correlation function, one would obtain an expression
$$ \large
    \propto \int_{-\infty}^{\infty} \dif{t} \, e^{-\ii t (\omega - \bar{E}_n)}, 
$$
which is unfortunately not well defined as a function, but only in a distributional sence. We will remedy this problem in measurability in a twofold way.
First, we will restrict the time range to the interval $[0, \infty)$.
From a physical point of few this implies that we are only interested in the correlation of the system were Operator $\hat{A}$ is applied after operator $\hat{B}$ was applied. 
One calls this the **causal Green's function** 

$$ \large
    G_{AB}(t) \coloneqq \Theta(t) \mel{\psi_g}{e^{\ii t \hat{H}} \hat{A} e^{-\ii t \hat{H}}  \hat{B}}{\psi_g} = \Theta(t) \sum_n e^{-\ii t \bar{E}_n} \mel{\psi_g}{ \hat{A} }{n}\mel{n} {\hat{B}}{\psi_g}.
$$,
where $\Theta(t)$ is the Heaviside step function.

The second step is to promote $\omega$ to a complex number, i.e. $\omega \to \Omega \coloneqq \omega + \ii \delta$. We can therefore evaluate the Fourier transformation of the causal Green's function as

$$ \large
    \int_{-\infty}^{\infty} \dif{t} \, \Theta(t) \, e^{-\ii t (\omega - \bar{E}_n)} =  \int_{0}^{\infty} \dif{t} \, e^{-\ii t (\omega - \bar{E}_n)} = \frac{1}{\omega + \ii \delta - \bar{E}_n} \quad \mathrm{for} \quad \delta > 0.
$$

To finally obtain the Fourier transform of the causal Green's function we have to take the limit $\delta \to 0$ from above, which gives (keeping in mind, that a distribution is only defined via integration by a test function)

$$ \large
    \lim_{\delta \to 0} \frac{1}{\omega + \ii \delta - \bar{E}_n} = \mathrm{P.V.} \left(\int_{-\infty}^{\infty} \dif{\omega} \,\frac{1}{\omega - \bar{E}_n}\right) - \ii \pi \delta(\omega - \bar{E}_n),
$$

where $\mathrm{P.V.}$ denotes the principal value of an integral. 
This results in our final result for the Fourier transform of the causal Green's function

$$ \large
	\tilde{G}_{AB}(\omega) \coloneqq \expval{\hat{A}\hat{B}}_\omega \coloneqq \sum_n \mel{\psi_g}{ \hat{A} }{n}\mel{n} {\hat{B}}{\psi_g} \left( \mathrm{P.V.} \left(\int_{-\infty}^{\infty} \dif{\omega} \,\frac{1}{\omega - \bar{E}_n}\right) - \ii \pi \delta(\omega - \bar{E}_n)\right).
$$

## Numerical Approximation of $\tilde{G}_{AB}(\omega)$

Since neither the principal value nor the delta distribution are well defined in a numerical sense, to visualize them we have to approximate them by using **Lorentzian curves** for the real and imaginary part of $\tilde{G}_{AB}(\omega)$. 
The Lorentzian curves are defined as
$$
\large
\begin{align*} 
	\im{\tilde{G}_{AB}(\omega)} &\approx \sum_n  \mel{\psi_g}{ \hat{A} }{n}\mel{n} {\hat{B}}{\psi_g} \left( \frac{\delta / 2}{(\omega - \bar{E}_n)^2 + \delta^2} \right) \\
	%
	\re{\tilde{G}_{AB}(\omega)} &\approx \sum_n  \mel{\psi_g}{ \hat{A} }{n}\mel{n} {\hat{B}}{\psi_g} \left( \frac{\bar{E}_n - \omega }{(\omega - \bar{E}_n)^2 + \delta^2} \right),
\end{align*}
$$
where $\delta \ll 1$ is a small parameter that assures finite width of the Lorentzian curves.

In [ ]:
# create instance of DynamicalHubbard class
h1 = dh.DynamicalHubbard()

# layout of widgets
box_layout = widgets.Layout(border='solid 2px')

## Spin-Spin Correlation Functions

In the two widgets below you can explore the real an imaginary part of the Fourier transform of the causal Green's function for the Hubbard model. Due to symmetry reasons, some combinations of $i$ and $j$ are equivalent, e.g. for $n=6$ we have $\expval{\hat{S_1}\hat{
S_2}}_\omega = \expval{\hat{S_1}\hat{S_6}}_\omega$, which is why we only show the minimal subset containing all different combinations

By changing the value of $U$ the spectral weight is shifted to higher energies, while the width of the Lorentzian curves is determined by the value of $\delta$. Setting $\delta = 0$ would result in a delta distribution, which is not well-defined in a numerical sense, and will instead set $\delta = 10^{-5}$.

One can adjust the range of $\omega$ via the range slider, which allows plotting over negative values of $\omega$. Due to the symmetry of the Hubbard model, the spectral weight is symmetric around $\omega = 0$, which can be explored by the user.

### Real-Space

In [ ]:
close_widgets(dir())

#create the widget
w1 = widgets.interactive(lambda **kwargs: h1.Plot_G_SzSz(Lorentzian="Imaginary"), w=h1.omega_range, u=h1.u25, d=h1.delta, n=h1.n, s_up=h1.s_up, s_down=h1.s_down, box=h1.t_ij);

# create the save figure button
filename = set_filename("Dynamical_Spin-Spin_Correlation_Imaginary.pdf")
Save_Figure_Button.on_click(functools.partial(Click_Save_Figure, widget=w1, name_widget=filename, output=h1.out, path="Figures/"))

# vertical and horizontal box for widgets
d1 = VBox([
    	HBox([Save_Figure_Button, filename, h1.out], layout=box_layout),
        HBox([
            VBox(w1.children[0:3], layout=box_layout),
			VBox(w1.children[3:6], layout=box_layout),
			VBox([w1.children[6]], layout=box_layout)
			]),
        w1.children[-1],
		])

display(d1)
w1.update()	# otherwise graph will only appear after the first interaction

In [ ]:
close_widgets(dir())

#create the widhet
w2 = widgets.interactive(lambda **kwargs: h1.Plot_G_SzSz(Lorentzian="Real"), w=h1.omega_range, u=h1.u25, d=h1.delta, n=h1.n, s_up=h1.s_up, s_down=h1.s_down, box=h1.t_ij);

# create the save figure button
filename = set_filename("Dynamical_Spin-Spin_Correlation_Real.pdf")
Save_Figure_Button.on_click(functools.partial(Click_Save_Figure, widget=w2, name_widget=filename, output=h1.out, path="Figures/"))

# vertical and horizontal box for widgets
d2 = VBox([
    	HBox([Save_Figure_Button, filename, h1.out], layout=box_layout),
        HBox([
            VBox(w2.children[0:3], layout=box_layout),
			VBox(w2.children[3:6], layout=box_layout),
			VBox([w2.children[6]], layout=box_layout)
			]),
        w2.children[-1],
		])

display(d2)
w2.update()	# otherwise graph will only appear after the first interaction

### Reciprocal-Space

Like in the previous notebook we can use translation symmetry to reduce the number of different real-space correlation functions ($n^2$) we have to calculate to just $\left \lfloor n/2 \right \rfloor$ non-zero, reciprocal-space functions, holding all the information. Of course one can switch back and forth from one representation to the other via a Fourier transform.

The widget below shows the imaginary and real part of all relevant time-dependent, reciprocal-space correlation functions $\expval{S_z(\vb k) S_z(-\vb k)}_\omega$ as a function of the interaction strength $U$. Note that in this case we have no symmetry around $\omega = 0$. Also, for $k=0 \cdot \pi$ the correlation function is identically zero.

In [ ]:
close_widgets(dir())

#create the widhet
w3 = widgets.interactive(h1.Plot_G_SzkSzk, w=h1.omega_range, u=h1.u25, d=h1.delta, n=h1.n, s_up=h1.s_up, s_down=h1.s_down, box=h1.t_ij);

# create the save figure button
filename = set_filename("Reciprocal_Dynamical_Spin-Spin_Correlation.pdf")
Save_Figure_Button.on_click(functools.partial(Click_Save_Figure, widget=w3, name_widget=filename, output=h1.out, path="Figures/"))

# vertical and horizontal box for widgets
d3 = VBox([
    	HBox([Save_Figure_Button, filename, h1.out], layout=box_layout),
        HBox([
            VBox(w3.children[0:3], layout=box_layout),
			VBox(w3.children[3:6], layout=box_layout),
			VBox([w3.children[6]], layout=box_layout)
			]),
        w3.children[-1],
		])

display(d3)
w3.update()	# otherwise graph will only appear after the first interaction

## Electronic Propagators

In [1]:
%matplotlib inline


#TODO: Rename module shortcuts such as hb into something usefull after development
import Modules.M05_Dynamical_Correlations.Module_Dynamical_Hubbard as dh
from Modules.M00_General.Module_Widgets_and_Sliders import Save_Figure_Button, Click_Save_Figure, set_filename

# default Jupyter widgets
import ipywidgets as widgets
from ipywidgets import HBox, VBox

# Plotting
import seaborn as sns
sns.set_style("darkgrid")
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size':14})

# for printlenght and save figures button
import functools
import numpy as np
np.set_printoptions(linewidth=150) #set output length, default=75

def close_widgets(DIR) -> None:
    """Close all widgets `wi` and displays `di` in notebook directory `dir()`.
    Also clear `Save_Figure_Button`
    """
    for i in range(100):
        if f"w{i}" in DIR:
            exec(f"w{i}.close()")
        if f"d{i}" in DIR:
            exec(f"d{i}.close()")
    # clear `Save_Figure_Button` callbacks, otherwise all previous callbacks are executed and figures are saved multiple times
    Save_Figure_Button._click_handlers.callbacks = []
            

%load_ext autoreload
%autoreload 2

In [2]:
hp = dh.DynamicalHubbardPropagator()
close_widgets(dir())

In [3]:
display(hp.n, hp.s_up, hp.s_down, hp.out)

BoundedIntText(value=6, description='$n = $', layout=Layout(width='3cm'), max=7, min=2, style=DescriptionStyle…

BoundedIntText(value=3, description='$s_\\mathrm{up} = $', layout=Layout(width='5cm'), max=6, style=Descriptio…

BoundedIntText(value=3, description='$s_\\mathrm{down} = $', layout=Layout(width='5cm'), max=6, style=Descript…

Output()

In [23]:
# %%timeit
hp.Op_Ck_dagger(0).shape

(1225, 1225)

In [ ]:
ep, vp = hp.hubbard_plus.All_Eigvals_and_Eigvecs
em, vm = hp.hubbard_minus.All_Eigvals_and_Eigvecs

In [ ]:
print(ep[4])
print(vp[4])
print(np.round(em[4], 3))
print(np.round(vm[4], 3))


In [ ]:
print(hp.hubbard_plus.basis.shape)
print(hp.hubbard_minus.basis.shape)
# print(hp.hubbard_plus.basis)
# print(hp.hubbard_minus.basis)
print(hp.hubbard_plus.Ht)
print(hp.hubbard_minus.Ht)

In [ ]:
hp.out